In [121]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px

In [122]:
df = pd.read_csv("data/Speed_Dating_Data.csv", encoding="latin-1")

0. Taux de match par wave

In [ ]:
# Taux de match par wave
taux_match_wave = df.groupby("wave")["match"].mean().reset_index()
taux_match_wave["match"] = (taux_match_wave["match"] * 100).round(1)

# tri décroissant
taux_match_wave = taux_match_wave.sort_values("match", ascending=False)
taux_match_wave["wave"] = taux_match_wave["wave"].astype(str)


In [ ]:
# Visualisation
taux_match_fig = px.bar(
    taux_match_wave,
    x="wave",
    y="match",
    text="match",
    category_orders={"wave": taux_match_wave["wave"].tolist()},
    labels={"wave": "Wave", "match": "Taux de match (%)"},
    title="Taux de match par wave (ordre décroissant)"
)

taux_match_fig.update_traces(texttemplate="%{text} %", textposition="outside")
taux_match_fig.show()

1. What are the least desirable attributes in a male partner? Does this differ for female partners?

Variables :
- attr, sinc, intel, fun, amb, shar → notes données aux partenaires pendant les speed dates.
- gender (0=femme, 1=homme).

In [126]:
# Colonnes utiles
attrs = ["attr","sinc","intel","fun","amb","shar"]

In [127]:
# Sexe du partenaire
df["partner_gender"] = 1 - df["gender"]

# Calcul des moyennes par attribut et sexe du partenaire
means = df.groupby("partner_gender")[attrs].mean().T
means.columns = ["Partenaire féminin", "Partenaire masculin"]

attrs_labels = {
    "attr": "Attractivité",
    "sinc": "Sincérité",
    "intel": "Intelligence",
    "fun": "Humour",
    "amb": "Ambition",
    "shar": "Intérêts communs"
}

means = means.rename(index=attrs_labels)

In [128]:

# Visualisation
plot_df = means.reset_index().melt(id_vars="index", var_name="Sexe du partenaire", value_name="Note moyenne")
plot_df.rename(columns={"index": "Attribut"}, inplace=True)

attrs_part_fig = px.bar(
    plot_df, 
    x="Attribut", 
    y="Note moyenne", 
    color="Sexe du partenaire",
    barmode="group", 
    title="Notes moyennes par attribut et sexe du partenaire",
    color_discrete_map = {"Partenaire masculin":"#1f77b4", "Partenaire féminin":"#ff7f0e"}
    )

attrs_part_fig.update_traces(
    texttemplate="%{y:.2f}", 
    textposition="inside"
    )

attrs_part_fig.show()

2. How important do people think attractiveness is in potential mate selection vs. its real impact?

Variables :
- attr1_1, sinc1_1, intel1_1, fun1_1, amb1_1, shar1_1 : importance déclarée avant l’événement
- attr7_2 (ou attr7_3) : importance réelle déclarée après coup
- attr (notes données pendant les dates) : importance révélée dans le comportement

In [129]:
df = pd.read_csv("data/Speed_Dating_Data.csv", encoding="latin-1")

In [130]:
# colonnes utiles
cols = ["attr1_1", "attr7_2", "attr", "dec"]
df = df[cols].dropna()

In [131]:
# Importance déclarée (avant et après)
declared_means = {
    "Avant événement (attr1_1)": df["attr1_1"].mean(),
    "Après événement (attr7_2)": df["attr7_2"].mean()
}

In [132]:
print("Importance déclarée de l'attrait :")
for k, v in declared_means.items():
    print(f"  {k} : {v:.2f}")

Importance déclarée de l'attrait :
  Avant événement (attr1_1) : 23.85
  Après événement (attr7_2) : 32.74


In [143]:
# Bar chart des importances déclarées
declared_df = pd.DataFrame(list(declared_means.items()), columns=["Temporalité", "Importance moyenne"])

fig1 = px.bar(
    declared_df,
    x="Temporalité", y="Importance moyenne",
    text="Importance moyenne",
    labels={"Temporalité":"Temporalité", "Importance moyenne":"Importance moyenne"},
    title="Importance déclarée de l'attrait (avant vs après)"
)
fig1.update_traces(texttemplate="%{text:.1f}", textposition="outside", marker_color=["#8ecae6","#219ebc"])
fig1.update_layout(
    plot_bgcolor="white",
    xaxis_categoryorder="array",
    bargap=0.25
)
fig1.show()

In [134]:
# Impact réel : taux de 'oui' par note d'attrait
df["attr_round"] = df["attr"].round().astype(int)
rate = df.groupby("attr_round")["dec"].mean().reset_index()

# Visualisation
rate_fig = px.line(
    rate, 
    x="attr_round", 
    y="dec", 
    markers=True,
    title="Taux de oui selon la note d'attractivité (impact réel)",
    labels={"attr_round":"Note d'attractivité","dec":"Taux de oui"}
)

rate_fig.update_yaxes(tickformat=".0%", rangemode="tozero")

rate_fig.show()

3. Are shared interests more important than a shared racial background?

Variables :
- int_corr (corrélation entre les centres d’intérêt des deux partenaires)
- samerace (1 = même race, 0 = non)
- match 

In [135]:
df = pd.read_csv("data/Speed_Dating_Data.csv", encoding="latin-1")

In [136]:
# proportion de match selon le critère ethnique
race_match = df.groupby(["samerace", "gender"])["match"].mean().reset_index()

gender_labels = {0: "Femme", 1: "Homme"}
race_labels = {0: "Origine différente", 1: "Même origine"}

race_match["gender_label"] = race_match["gender"].map(gender_labels)
race_match["samerace_label"] = race_match["samerace"].map(race_labels)

In [137]:
# Visualisation
race_match_fig = px.bar(
    race_match,
    x="samerace_label",
    y="match",
    color="gender_label",
    barmode="group",
    text="match",
    labels={"samerace_label": "Origine raciale", "match": "Taux de match", "gender_label": "Sexe"},
    title="Taux de match selon l'origine partagée, par sexe",
    color_discrete_map={"Femme": "#ff7f0e", "Homme": "#1f77b4"},
)

race_match_fig.update_traces(texttemplate="%{text:.1%}", textposition="inside")
race_match_fig.update_yaxes(tickformat=".0%", rangemode="tozero")

race_match_fig.update_layout(
    plot_bgcolor="white",
    xaxis_categoryorder="array",
    xaxis_categoryarray=["Origine différente", "Même origine"],
    bargap=0.25
)

race_match_fig.show()

In [138]:
# relation entre intérêts partagés et match
df_plot = df.copy()
df_plot["match_label"] = df_plot["match"].map({0: "Pas de match", 1: "Match"})
df_plot["gender_label"] = df_plot["gender"].map({0: "Femme", 1: "Homme"})
mean_vals = df_plot.groupby(["match_label", "gender_label"])["int_corr"].mean().reset_index()

corr_match_fig = px.violin(
    df_plot,
    x="match_label",
    y="int_corr",
    color="match_label",
    box=True,
    labels={"int_corr": "Corrélation d'intérêts", "match_label": "Résultat"},
    title="Corrélation des intérêts selon le résultat du rendez-vous"
)

corr_match_fig.show()

In [139]:
# relation entre intérêts partagés et match
corr_match_fig = fig = px.bar(
    mean_vals,
    x="match_label",
    y="int_corr",
    color="gender_label",
    barmode="group",
    labels={"match_label":"Résultat","int_corr":"Corrélation moyenne","gender_label":"Genre"},
    title="Corrélation des intérêts selon le résultat (par genre)",
    color_discrete_map={"Femme": "#ff7f0e", "Homme": "#1f77b4"}
)

corr_match_fig.update_traces(
    texttemplate="%{y:.3f}", 
    textposition="inside"
)

corr_match_fig.update_yaxes(
    tickformat=".0%", 
    rangemode="tozero"
)

corr_match_fig.update_layout(
    plot_bgcolor="white",
    xaxis_categoryorder="array",
    bargap=0.25
)

corr_match_fig.show()

4. Can people accurately predict their own perceived value in the dating market?

Variables : 
- expnum
- Match
- match_es (attentes) vs nombre réel de match

In [140]:
# warning : 551 participants uniques (iid) mais moins de points dans le scatter car 
# lié à des valeurs manquantes (NaN) dans expnum ou real_matches

df["real_matches"] = df.groupby("iid")["match"].transform("sum")

predict_fig = px.scatter(
    df.drop_duplicates("iid"), 
    x="expnum", 
    y="real_matches",
    size = "real_matches",
    color="real_matches", 
    color_continuous_scale="plasma",
    size_max=40, 
    labels={"expnum": "Nombre attendu", "real_matches": "Nombre réel"}, 
    title="Attentes vs Réalité des matchs",
    hover_data=["iid"]
)

predict_fig.show()

5. In terms of getting a second date, is it better to be someone's first speed date of the night or their last?

Variables : 
- order (rang du rendez-vous dans la soirée)
- match

In [141]:
# les premiers rendez-vous matchent plus = effet “fraîcheur” ou motivation ?
# le taux baisse vers 14 ou 15  = fatigue décisionnelle ou comparaison plus sévère ?
# une remontée possible à la fin = effet “dernier choix” ou relâchement = ou why not ?

order_match = df.groupby("order")["match"].mean().reset_index().round(2)

order_match_fig = px.scatter(
    order_match,
    x="order", 
    y="match",
    trendline="lowess",
    size = "match",
    color = "match", 
    color_continuous_scale="Viridis",
    size_max=20,
    labels={"order": "Ordre du rendez-vous", "match": "Taux de match"}
)

order_match_fig.show()